In [2]:
'''
problem: predict the rate of resource consumption among 50 states in the US and Columbia
        in order to find the relavance between the amount of fuel consumed and fuel tax

input: observations in 7 attributes - drivers -> dlic, fuelC -> fuel, income, miles -> logMiles, pop, tax, state
output: the predicted amount of fuel
solution: linear regression with multiple variables
'''

'\nproblem: predict the rate of resource consumption among 50 states in the US and Columbia\n        in order to find the relavance between the amount of fuel consumed and fuel tax\n\ninput: observations in 7 attributes - drivers -> dlic, fuelC -> fuel, income, miles -> logMiles, pop, tax, state\noutput: the predicted amount of fuel\nsolution: linear regression with multiple variables\n'

read data from text

In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt

In [4]:
with open('fuel.txt') as f:
    lines = f.readlines()

x_data = []
y_data = []
lines.pop(0)

for line in lines:
    tokens = line.strip().split(',')
    tokens.pop(0)
    tokens = list(map(float, tokens))
    fuel = 1000 * tokens[1] / tokens[5]
    dlic = 1000 * tokens[0] / tokens[5]
    logMiles = math.log2(tokens[3])
    y_data.append([fuel])
    x_data.append([tokens[-1], dlic, tokens[2], logMiles])

x_data = np.asarray(x_data)
y_data = np.asarray(y_data)

##### QR decomposition using Householder algorithm

In [5]:
def qr_householder(A):
    """ 
    Compute QR decomposition of A using Householder reflection
    """
    M = A.shape[0]
    N = A.shape[1]

    # set Q to the identity matrix
    Q = np.identity(M)

    # set R to zero matrix
    R = np.copy(A)

    for n in range(N):
        # vector to transform
        x = A[n:, n]
        k = x.shape[0]

        # compute ro=-sign(x0)||x||
        ro = -np.sign(x[0]) * np.linalg.norm(x)

        # compute the householder vector v
        e = np.zeros(k)
        e[0] = 1
        v = (1 / (x[0] - ro)) * (x - (ro * e))

        # apply v to each column of A to find R
        for i in range(N):
            R[n:, i] = R[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ R[n:, i])

        # apply v to each column of Q
        for i in range(M):
            Q[n:, i] = Q[n:, i] - (2 / (v@v)) * ((np.outer(v, v)) @ Q[n:, i])

    return Q.transpose(), R

In [16]:
def linear_regression(x_data, y_data):
    """

    This function calculate linear regression base on x_data and y_data
    :param x_data: vector
    :param y_data: vector
    :return: w (regression estimate)
    """

    # add column 1
    x_bars = np.concatenate((np.ones((x_data.shape[0], 1)), x_data), axis=1)

    Q, R = qr_householder(x_bars) # QR decomposition
    R_pinv = np.linalg.pinv(R) # calculate inverse matrix of R
    A = np.dot(R_pinv, Q.T) # apply formula

    return np.dot(A, y_data)

In [17]:
w = linear_regression(x_data, y_data) # get result
w = w.T.tolist()
line = ['Intercept', 'Tax', "Dlic", "Income", 'LogMiles']
res = list(zip(line, w[0]))
for o in res:
    print("{: >20}: {: >10}".format(*o))

           Intercept: 154.1928445773105
                 Tax: -4.22798320832962
                Dlic: 0.4718712134419811
              Income: -0.0061353309704176
            LogMiles: 18.545274506047992


In [12]:
x_data.shape

(51, 4)

##### using scikit-learn library

In [9]:
from sklearn import datasets, linear_model
# load training data here and assign to Xbar (obs. Data) and y (label)
# x_bars = np.concatenate((np.ones((x_data.shape[0], 1)), x_data), axis=1)
# print(x_bars[0])

# fit the model by Linear Regression
regr = linear_model.LinearRegression(fit_intercept=True)
# fit_intercept = False for calculating the bias
regr.fit(x_data, y_data)
# regr.fit(x_bars, y_data) # fit_intercept=False

LinearRegression()

In [10]:
w = np.insert(regr.coef_, 0, regr.intercept_)
w = w.tolist()
line = ['Intercept', 'Tax', "Dlic", "Income", 'LogMiles']
res = list(zip(line, w))
for o in res:
    print("{: >20}: {: >10}".format(*o))

           Intercept: 154.1928445773035
                 Tax: -4.227983208329684
                Dlic: 0.47187121344198685
              Income: -0.006135330970417716
            LogMiles: 18.54527450604802


visualizing